<a href="https://colab.research.google.com/github/supertime1/BP_PPG/blob/master/Data%20Processing/Biometric_Info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Introduction

This notebook is to generate biometric information of patients in wfdb matched dataset

#2. Setup Environment

In [ ]:
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext tensorboard
import numpy as np
import os
import shutil
import glob
import wfdb
from wfdb import processing
from datetime import datetime
import sklearn.metrics
import itertools
import io
import pickle

#3.Extract patient information from MIMIC-III

In [ ]:
#extract subject ID and signal recording date from header files
date_ls = []
id_ls = []
for name in glob.glob("D:/WFDB//matched/p*/*/p*[!n].hea"):
  idx = name.index('-')
  r_date = name[idx+1:idx+11]
  s_id = name.split('\\')[2]
  date_ls.append(r_date)
  id_ls.append(s_id)

#create a matched dataframe to store subject ID and signal recording date
d = {'SUBJECT_ID': id_ls, 'Receive Date': date_ls}
matched_patient = pd.DataFrame(data = d)
print("There are total ", len(date_ls), "header files")

matched_patient.to_csv("C:/Users/57lzhang.US04WW4008/Desktop/Blood pressure/BP data/PATIENT_MATCHED.csv")

There are total  22317 header files


In [ ]:
#import PATIENTS file with DOB and GENDER info
directory = "C:/Users/57lzhang.US04WW4008/Desktop/Blood pressure/BP data/PATIENTS.csv"
bio =pd.read_csv(directory,encoding= 'unicode_escape')
#inner join two dataframes
new = bio.merge(matched_patient, on='SUBJECT_ID', how='inner')

There are total  22317 header files


In [ ]:
#calculate subject age by substracting DOB from receive date
ages = []
for i in range(len(new)):
  age = (pd.to_datetime(new['Receive Date'][i]) - pd.to_datetime(new['DOB'][i])).days//365
  ages.append(age)

In [ ]:
pd.to_datetime(new['Receive Date'][16927])

Timestamp('2125-12-03 00:00:00')

In [ ]:
pd.to_datetime(new['DOB'][16927])

Timestamp('1825-12-03 00:00:00')

In [ ]:
bio['DOB'][bio['SUBJECT_ID']=='p075764']

40127    1825-12-03 00:00:00
Name: DOB, dtype: object

In [ ]:
dic = {'SUBJECT_ID': new['SUBJECT_ID'], 'AGE': ages, 'GENDER': new['GENDER']}
bio_patient = pd.DataFrame(data = dic)
bio_patient.to_csv("C:/Users/57lzhang.US04WW4008/Desktop/Blood pressure/BP data/BIO_PATIENTS.csv")

In [ ]:
#to get rid of the typos in MIMIC-III, where some patients have age >300 

In [ ]:
#read subject ID from cleaned dataset(train, val, and test)